In [1]:
import pydgraph
import pandas as pd
import tqdm
import os


filepath = './common-crawl/' + os.listdir('./common-crawl/')[0]
df = pd.read_csv(filepath, nrows=1000, compression='gzip', usecols=['asn_num', 'domain', 'ip', 'country', 'asn_org'])
df.head(10)

,domain,ip,asn_num,country,asn_org
0,aice-interpretes.com,89.140.72.171,6739,ES,ONO-AS Cableuropa - ONO
1,aice-interpretes.com,89.140.72.171,6739,ES,ONO-AS Cableuropa - ONO
2,aice-interpretes.com,89.140.72.171,6739,ES,ONO-AS Cableuropa - ONO
3,aice-interpretes.com,89.140.72.171,6739,ES,ONO-AS Cableuropa - ONO
4,aice-interpretes.com,89.140.72.171,6739,ES,ONO-AS Cableuropa - ONO
5,aice-interpretes.com,89.140.72.171,6739,ES,ONO-AS Cableuropa - ONO
6,aice-interpretes.com,89.140.72.171,6739,ES,ONO-AS Cableuropa - ONO
7,aice-interpretes.com,89.140.72.171,6739,ES,ONO-AS Cableuropa - ONO
8,aice-interpretes.com,89.140.72.171,6739,ES,ONO-AS Cableuropa - ONO
9,aice-interpretes.com,89.140.72.171,6739,ES,ONO-AS Cableuropa - ONO


In [2]:
# Create a client.
def get_client():
    client_stub = pydgraph.DgraphClientStub('localhost:9080')
    return pydgraph.DgraphClient(client_stub)


# Drop All - discard all data and start from a clean slate.
def drop_all(client):
    return client.alter(pydgraph.Operation(drop_all=True))


# Set schema.
def set_schema(client):
    schema = """
    asnnum: int @index(int) .
    org: string .
    
    type ASN {
        asnnum
        org
    }
    
    domain: string @index(term,exact) .
    tld: string .
    country: string .
    ip: string .
    inasn: uid .
    
    type Domain {
        domain
        tld
        country
        ip
        inasn
    }
    
    """
    return client.alter(pydgraph.Operation(schema=schema))

client = get_client()
drop_all(client)
set_schema(client)

In [4]:
def insert(client, df):
    # Create a new transaction.
    txn = client.txn()
    try:
        # Create data.
        
        for n, row in tqdm.tqdm(enumerate(df.itertuples())):
            p = {
                'uid': '_:' + row.domain,
                'dgraph.type': 'Domain',
                'domain': row.domain,
                'tld': row.domain.split('.')[-1],
                'ip': row.ip,
                'inasn': [
                    {
                        'uid': '_:' + str(row.asn_num),
                        'dgraph.type': 'ASN',
                        'asnnum': row.asn_num,
                        'org': row.asn_org
                    }
                ],
            }

            # Run mutation.
            response = txn.mutate(set_obj=p)


        # Commit transaction.
        txn.commit()

    finally:
        # Clean up. Calling this after txn.commit() is a no-op and hence safe.
        txn.discard()
        
insert(client, df)

1000it [00:12, 83.26it/s]
